# Review ICCS'21
------
Author: Mikhail.Gasanov@skoltech.ru

In [ ]:
#Uncomment for Google Colab 


#!pip install pcse==5.4.2
#!pip install pymoo==0.4.2.1
#!git clone https://github.com/EDSEL-skoltech/multi_objective_irrigation.git
#%cd multi_objective_irrigation/



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import copy


from experiments.crop_model_en import Optimization

Platform not recognized, using system temp directory for PCSE settings.
Platform not recognized, using system temp directory for PCSE settings.


In [2]:
#colors for plots 

random_color_plot = '#fc8d59'
sugar_beet_color_plot = '#7b3294'
potato_color_plot = '#abd9e9'

## Farmer data

In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import json
import argparse
import multiprocessing
import matplotlib.pyplot as plt

WOFOST = Optimization()
path_to_user_file = './util/input_data/malino_sugar_beet.json'
with open(path_to_user_file, 'r') as f:
    WOFOST.user_parameters = json.load(f)
latitude = int(WOFOST.user_parameters['latitude'])
longitude = int(WOFOST.user_parameters['longitude'])
crop_name = WOFOST.user_parameters['crop_name']
path_CSV_dir = './data/meteo/'

def round_geoposition(x, prec=1, base=.5):
    return round(base * round(float(x)/base),prec)
latitude = round_geoposition(WOFOST.user_parameters['latitude'])
longitude = round_geoposition(WOFOST.user_parameters['longitude'])
WOFOST.weather_loader(path_CSV_dir, latitude, longitude)

WOFOST.data_dir = './util/input_data/'
crop_results=[]

# Range of year to compute mean crop yield and mean water loss
input_years = np.arange(WOFOST.NASA_last_year-20, WOFOST.NASA_last_year)
historical_yield, water_loss = [],[]
print('Sugar-beet\n')
for year in input_years:
    df_crop = WOFOST.crop_hpc(year)
    historical_yield.append(df_crop['TWSO'][-1])
    print('year', year, ', crop yield', "%.2f"% (df_crop['TWSO'][-1]/1000), 't/ha')
    water_loss.append(WOFOST.total_ammount_of_losed_water)
#Sugar-beet
df_sugar_beet = pd.DataFrame({"range": input_years,"yield":historical_yield, "loss":water_loss})

#Farmer Potato
WOFOST = Optimization()
path_to_user_file = './util/input_data/malino_potato.json'
with open(path_to_user_file, 'r') as f:
    WOFOST.user_parameters = json.load(f)

latitude = int(WOFOST.user_parameters['latitude'])
longitude = int(WOFOST.user_parameters['longitude'])
crop_name = WOFOST.user_parameters['crop_name']
path_CSV_dir = './data/meteo/'
latitude = round_geoposition(WOFOST.user_parameters['latitude'])
longitude = round_geoposition(WOFOST.user_parameters['longitude'])
WOFOST.weather_loader(path_CSV_dir, latitude, longitude)
WOFOST.data_dir = './util/input_data/'
crop_results=[]
input_years = np.arange(WOFOST.NASA_last_year-20, WOFOST.NASA_last_year)
historical_yield, water_loss = [], []

for year in input_years:
    df_crop = WOFOST.crop_hpc(year)
    historical_yield.append(df_crop['TWSO'][-1])
    print('year', year, ', crop yield', "%.2f"% (df_crop['TWSO'][-1]/1000), 't/ha')
    water_loss.append(WOFOST.total_ammount_of_losed_water)
df_potato = pd.DataFrame({"range": input_years,"yield":historical_yield, "loss":water_loss})

LOAD FROM LOCAL CSV WEATHER DATABASE
Sugar-beet

year 1999 , crop yield 12.04 t/ha
year 2000 , crop yield 10.64 t/ha
year 2001 , crop yield 13.24 t/ha
year 2002 , crop yield 9.94 t/ha
year 2003 , crop yield 10.99 t/ha
year 2004 , crop yield 11.28 t/ha
year 2005 , crop yield 13.63 t/ha
year 2006 , crop yield 10.99 t/ha
year 2007 , crop yield 12.18 t/ha
year 2008 , crop yield 10.74 t/ha
year 2009 , crop yield 12.88 t/ha
year 2010 , crop yield 7.59 t/ha
year 2011 , crop yield 13.61 t/ha
year 2012 , crop yield 14.77 t/ha
year 2013 , crop yield 13.56 t/ha
year 2014 , crop yield 13.21 t/ha
year 2015 , crop yield 13.17 t/ha
year 2016 , crop yield 12.99 t/ha
year 2017 , crop yield 9.48 t/ha
year 2018 , crop yield 13.06 t/ha
LOAD FROM LOCAL CSV WEATHER DATABASE
year 1999 , crop yield 9.43 t/ha
year 2000 , crop yield 14.01 t/ha
year 2001 , crop yield 13.02 t/ha
year 2002 , crop yield 7.83 t/ha
year 2003 , crop yield 14.90 t/ha
year 2004 , crop yield 14.07 t/ha
year 2005 , crop yield 15.08 t/ha
y

In [13]:
from pymoo.factory import get_problem, get_visualization, get_decomposition

weights = np.array([0.5, 0.5])

NSGA_potato_yield_review, NSGA_potato_water_loss_review = [],[]

Random_potato_yield_review, Random_potato_water_loss_review = [], []

NSGA_sugar_beet_yield_review, NSGA_sugar_beet_water_loss_review = [], []

Random_sugar_beet_yield_review, Random_sugar_beet_water_loss_review = [],[]

for i in range(1,11):
    path_to_folder_with_data = os.path.join('./review_pymoo/review_npy_'+str(i))

    #Sugar beet NSGA-II
    sugar_beet_crop_yield = np.load(path_to_folder_with_data+'/sugar_beet_crop_yields.npy')
    sugar_beet_irrigation_amount = np.load(path_to_folder_with_data+'/sugar_beet_irrigation_ammount.npy')
    sugar_beet_water_loss = np.load(path_to_folder_with_data+'/sugar_beet_water_loss.npy')
    sugar_beet_paretto_front = np.load(path_to_folder_with_data+'/sugar_beet_function_values_for_paretto.npy')
    sugar_beet_optimal_solution = np.load(path_to_folder_with_data+'/sugar_beet_optimal_solutions.npy',allow_pickle=True)

    # Potato NSGA-II
    potato_crop_yield = np.load(path_to_folder_with_data+'/potato_crop_yields.npy')
    potato_irrigation_amount = np.load(path_to_folder_with_data+'/potato_irrigation_ammount.npy')
    potato_water_loss = np.load(path_to_folder_with_data+'/potato_water_loss.npy')
    potato_paretto_front = np.load(path_to_folder_with_data+'/potato_function_values_for_paretto.npy')
    potato_optimal_solution = np.load(path_to_folder_with_data+'/potato_optimal_solutions.npy',allow_pickle=True)

    #Sugar beet Monte-Carlo
    
    path_to_folder_with_data = os.path.join('./review_pymoo/', 'random_review', 'random_search_'+str(i))
    random_crop_yield_sugar_beet = np.load(os.path.join(path_to_folder_with_data, 'sugar_beet_crop_yields.npy'))
    random_water_loss_sugar_beet = np.load(os.path.join(path_to_folder_with_data, 'sugar_beet_water_loss.npy'))
    random_irrigation_sugar_beet = np.load(os.path.join(path_to_folder_with_data, 'sugar_beet_irrigation_ammount.npy'))
    sugar_beet_random = np.stack([random_crop_yield_sugar_beet, random_water_loss_sugar_beet], axis=1)

    #Potato Monte-Carlo
    random_crop_yield_potato = np.load(os.path.join(path_to_folder_with_data, 'potato_crop_yields.npy'))
    random_water_loss_potato = np.load(os.path.join(path_to_folder_with_data, 'potato_water_loss.npy'))
    random_irrigation_potato = np.load(os.path.join(path_to_folder_with_data, 'potato_irrigation_ammount.npy'))
    potato_random = np.stack([random_crop_yield_potato, random_water_loss_potato], axis=1)



    #Load and normalize paretto front fot potato crop
    path_to_folder_with_data = os.path.join('./review_pymoo/review_npy_'+str(i))
    potato_paretto_front = np.load(path_to_folder_with_data+'/potato_function_values_for_paretto.npy')
    normalized_potato_paretto_front = copy.deepcopy(potato_paretto_front)
    normalized_potato_paretto_front[:, 0] = (potato_paretto_front[:, 0]-np.mean(potato_paretto_front[:, 0]))/np.std(potato_paretto_front[:, 0])
    normalized_potato_paretto_front[:, 1] = (potato_paretto_front[:, 1]-np.mean(potato_paretto_front[:, 1]))/np.std(potato_paretto_front[:, 1])


    # Return values of crop to original values
    normalized_potato_paretto_front[:, 0] = normalized_potato_paretto_front[:, 0] * -1

    decomp = get_decomposition("weighted-sum")
    I_paretto_potato = get_decomposition("weighted-sum").do(normalized_potato_paretto_front, weights).argmin()    
    NSGA_potato_yield_review.append(potato_paretto_front[I_paretto_potato][0])
    NSGA_potato_water_loss_review.append(potato_paretto_front[I_paretto_potato][1])
    
    # Load and normalize Monte-Carlo data for potato crop
    normalized_potato_random_search = copy.deepcopy(potato_random)


    normalized_potato_random_search[:,0]=(potato_random[:,0] - np.mean(potato_random[:,0]))/np.std(potato_random[:,0])
    normalized_potato_random_search[:,1]=(potato_random[:,1] - np.mean(potato_random[:,1]))/np.std(potato_random[:,1])

    normalized_potato_random_search[:,0] = normalized_potato_random_search[:,0] * -1

    decomp = get_decomposition("weighted-sum")
    I_random_potato = get_decomposition("weighted-sum").do(normalized_potato_random_search, weights).argmin()
    Random_potato_water_loss_review.append(potato_random[I_random_potato][1])
    Random_potato_yield_review.append(potato_random[I_random_potato][0])

    path_to_folder_with_data = os.path.join('./review_pymoo/review_npy_'+str(i))
    sugar_beet_paretto_front = np.load(path_to_folder_with_data+'/sugar_beet_function_values_for_paretto.npy')
    normalized_sugar_beet_paretto_front = copy.deepcopy(sugar_beet_paretto_front)
    normalized_sugar_beet_paretto_front[:, 0] = (sugar_beet_paretto_front[:, 0]-np.mean(sugar_beet_paretto_front[:, 0]))/np.std(sugar_beet_paretto_front[:, 0])
    normalized_sugar_beet_paretto_front[:, 1] = (sugar_beet_paretto_front[:, 1]-np.mean(sugar_beet_paretto_front[:, 1]))/np.std(sugar_beet_paretto_front[:, 1])


    # Return values of crop to original values
    normalized_sugar_beet_paretto_front[:, 0] = normalized_sugar_beet_paretto_front[:, 0] * -1

    decomp = get_decomposition("weighted-sum")
    I_paretto_sugar_beet = get_decomposition("weighted-sum").do(normalized_sugar_beet_paretto_front, weights).argmin()

    NSGA_sugar_beet_water_loss_review.append(sugar_beet_paretto_front[I_paretto_sugar_beet][1])
    NSGA_sugar_beet_yield_review.append(sugar_beet_paretto_front[I_paretto_sugar_beet][0])
    # Load and normalize Monte-Carlo data for sugar-beet crop 

    normalized_sugar_beet_random_search = copy.deepcopy(sugar_beet_random)
    normalized_sugar_beet_random_search[:,0]=(sugar_beet_random[:,0] - np.mean(sugar_beet_random[:,0]))/np.std(sugar_beet_random[:,0])
    normalized_sugar_beet_random_search[:,1]=(sugar_beet_random[:,1] - np.mean(sugar_beet_random[:,1]))/np.std(sugar_beet_random[:,1])
    normalized_sugar_beet_random_search[:,0] = normalized_sugar_beet_random_search[:,0] * -1

    decomp = get_decomposition("weighted-sum")
    I_random_sugar_beet = get_decomposition("weighted-sum").do(normalized_sugar_beet_random_search, weights).argmin()

    Random_sugar_beet_water_loss_review.append(sugar_beet_random[I_random_sugar_beet][1])
    Random_sugar_beet_yield_review.append(sugar_beet_random[I_random_sugar_beet][0])
    
    #Plots

#     plt.figure(figsize=(13,8))
#     plt.scatter(potato_random[:,0], potato_random[:,1],facecolor=random_color_plot, s=10, alpha=0.07,marker='D' )


#     plt.scatter( potato_crop_yield, potato_water_loss, edgecolors='black', facecolor=potato_color_plot, s=10, alpha=0.07)

#     plt.scatter(potato_paretto_front[I_paretto_potato,0], potato_paretto_front[I_paretto_potato,1], edgecolors='black', facecolor=potato_color_plot, marker='o', s=130,  label = 'Ours')


#     # Real farmer solution
#     plt.scatter(potato_random[I_random_potato,0], potato_random[I_random_potato,1], edgecolors='black', facecolor=random_color_plot, marker='D', s=60, zorder=1, label='Monte-Carlo')
#     plt.scatter(df_potato['yield'].mean()/1000, df_potato['loss'].mean(), edgecolors='black',s = 60, marker='s', facecolor = 'green', label='Farmer solution')
    # plt.legend()

    # plt.savefig('./plots_ICCS/Random_search_and_optimizer_potato.png', dpi=300)
    # plt.savefig('./plots_ICCS/Random_search_and_optimizer_potato.svg' )

In [14]:
print(f'NSGA-II Potato - Yield - Mean: {np.mean(NSGA_potato_yield_review):.2f}, \
- Std: {np.std(NSGA_potato_yield_review):.2f}')
print(f'NSGA-II Potato - Water loss - Mean: {np.mean(NSGA_potato_water_loss_review):.2f}, \
- Std: {np.std(NSGA_potato_water_loss_review):.2f}')
print('\n')
print(f'Monte-Carlo Potato - Yield - Mean: {np.mean(Random_potato_yield_review):.2f}, \
- Std: {np.std(Random_potato_yield_review):.2f}')
print(f'Monte-Carlo Potato - Water loss - Mean: {np.mean(Random_potato_water_loss_review):.2f}, \
- Std: {np.std(Random_potato_water_loss_review):.2f}')
print('\n')

print(f'NSGA-II Sugar-beet - Yield - Mean: {np.mean(NSGA_sugar_beet_yield_review):.2f}, \
- Std: {np.std(NSGA_sugar_beet_yield_review):.2f}')
print(f'NSGA-II Sugar-beet - Water loss - Mean: {np.mean(NSGA_sugar_beet_water_loss_review):.2f}, \
- Std: {np.std(NSGA_sugar_beet_water_loss_review):.2f}')
print('\n')
print(f'Monte-Carlo Sugar-beet - Yield - Mean: {np.mean(Random_sugar_beet_yield_review):.2f}, \
- Std: {np.std(Random_sugar_beet_yield_review):.2f}')
print(f'Monte-Carlo Sugar-beet - Water loss - Mean: {np.mean(Random_sugar_beet_water_loss_review):.2f}, \
- Std: {np.std(Random_sugar_beet_water_loss_review):.2f}')

NSGA-II Potato - Yield - Mean: 14.11, - Std: 0.09
NSGA-II Potato - Water loss - Mean: 26.84, - Std: 1.40


Monte-Carlo Potato - Yield - Mean: 13.95, - Std: 0.18
Monte-Carlo Potato - Water loss - Mean: 34.54, - Std: 2.26


NSGA-II Sugar-beet - Yield - Mean: 12.16, - Std: 0.03
NSGA-II Sugar-beet - Water loss - Mean: 28.22, - Std: 0.98


Monte-Carlo Sugar-beet - Yield - Mean: 12.17, - Std: 0.03
Monte-Carlo Sugar-beet - Water loss - Mean: 42.38, - Std: 2.97
